# Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Se debe averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El informe tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Información-general." data-toc-modified-id="Información-general.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Información general.</a></span></li><li><span><a href="#Ejercicio-1.-Exploración-de-datos" data-toc-modified-id="Ejercicio-1.-Exploración-de-datos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ejercicio 1. Exploración de datos</a></span></li><li><span><a href="#Transformación-de-datos" data-toc-modified-id="Transformación-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Transformación de datos</a></span><ul class="toc-item"><li><span><a href="#Restaurar-valores-ausentes-en-total_income" data-toc-modified-id="Restaurar-valores-ausentes-en-total_income-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Restaurar valores ausentes en <code>total_income</code></a></span></li><li><span><a href="#Restaurar-valores-en-days_employed" data-toc-modified-id="Restaurar-valores-en-days_employed-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Restaurar valores en <code>days_employed</code></a></span></li></ul></li><li><span><a href="#Clasificación-de-datos" data-toc-modified-id="Clasificación-de-datos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clasificación de datos</a></span></li><li><span><a href="#Comprobación-de-las-hipótesis" data-toc-modified-id="Comprobación-de-las-hipótesis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comprobación de las hipótesis</a></span></li></ul></div>

## Información general. 


In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np 


In [2]:
# Carga los datos
data = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo



In [3]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos

data.shape

(21525, 12)

Existe un total de 21525 filas y 12 columnas en la base de datos a examinar. 

In [4]:
# vamos a mostrar las primeras filas N
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Se observa en la tabla que existe la necesidad de limpiar datos en algunas columnas como en días trabajados debido a la presencia de números negativos, la limpieza de las categorías de la columna educación donde se repiten sin embargo algunas se encuentran entre mayúsculas y minúsculas. Ademas la columna propósito contiene varias categorias parecidas pero sin estandarización. Adicional se necesita explorar si existen valores ausentes que afecten al estudio de crédito.

In [5]:
# Obtener información sobre los datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Se puede observar que existen valores ausentes en las columnas 'days_employed' y en 'total_income'.

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Se logra observar que existen valores simétricos entre las columnas de Días trabajados ('days_employed') y los ingresos totales ('total_income') debido a que si no ha trabajado no recibe ningun pago o remuneración salarial. 


In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
data[ (data['days_employed'].isnull()) & (data['total_income'].isnull())]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
number_filtered_rows = len(data[(data['days_employed'].isnull()) & (data['total_income'].isnull())])

total_value = (data.shape[0])
percentage_of_missing_values = (number_filtered_rows / total_value)

print(f'El porcentaje de valores ausentes es de{percentage_of_missing_values: .0%}')

El porcentaje de valores ausentes es de 10%


**Conclusión intermedia**

Al aplicar el condicional donde tanto los dias trabajados y el ingreso mensual contiene valores ausentes, observamos que la hipótesis de que si no se ha trabajado no se recibe remuneración salarial se comprueba. 

In [9]:
a = data[(data['days_employed'].isnull())]
b= data[(data['total_income'].isnull())]

print(a.shape[0])
print(b.shape[0])


2174
2174


Tanto en la columna 'days_employed' y en la columna 'total_income' existen 2174 datos ausentes. 

In [10]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

new_data = data[(data['days_employed'].isnull()) & (data['total_income'].isnull())]
data_grouped = new_data.groupby('children').agg({'children': 'count'})
print(data_grouped)
    


          children
children          
-1               3
 0            1439
 1             475
 2             204
 3              36
 4               7
 5               1
 20              9


Existe mayor cantidad de datos ausentes en la columna de días trabajados e ingresos para aquellas personas con 0 hijos. 

In [11]:
new_data['education'] = new_data['education'].str.lower()
data_grouped2 = new_data.groupby('education').agg({'education': 'count'})
print(data_grouped2)
    

                     education
education                     
bachelor's degree          544
primary education           21
secondary education       1540
some college                69


/tmp/ipykernel_43/1003960369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['education'] = new_data['education'].str.lower()


Existe ausencia de información en su mayoría para aquellas personas que tienen hasta un nivel de educación secundario.

In [12]:

data_grouped3 = new_data.groupby('family_status').agg({'family_status': 'count'})
print(data_grouped3)
    

                   family_status
family_status                   
civil partnership            442
divorced                     112
married                     1237
unmarried                    288
widow / widower               95


Existe mayor ausencia de información para aquellas personas casadas o en unión libre repecto a los días trabajados. 

In [13]:

data_grouped4 = new_data.groupby('income_type').agg({'income_type': 'count'})
print(data_grouped4)
    

               income_type
income_type               
business               508
civil servant          147
employee              1105
entrepreneur             1
retiree                413


Existe mayor cantidad de ausencia de información para personas que son empleados. 

El data frame original tiene el nombre de "data", mientras la data sin valores nulos se llamará "not_null_data"

In [14]:
not_null_data = data.dropna()
not_null_data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [15]:
not_null_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19351 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         19351 non-null  int64  
 3   education         19351 non-null  object 
 4   education_id      19351 non-null  int64  
 5   family_status     19351 non-null  object 
 6   family_status_id  19351 non-null  int64  
 7   gender            19351 non-null  object 
 8   income_type       19351 non-null  object 
 9   debt              19351 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           19351 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


Como se puede observar existen 19351 registros con datos completos, sin embargo, se debe continuar revisando la información interna de forma que sea lógica y apropiada para un buen análisis. 

Empezamos limpiando los datos de la educación debido a la diferencia en la forma de escritura (combinación de mayúsculas y minúsculas) alterando el análisis de la información provista.

In [16]:
not_null_data['education'] = not_null_data['education'].str.lower()

/tmp/ipykernel_43/3131316314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_null_data['education'] = not_null_data['education'].str.lower()


A continuación se realizará la comparación de 3 dataframes con el objetivo de identificar si el patrón de los datos ausentes tiene alguna relación con alguna de las variables de nuestro dataframe.
"data" - es la base original
"not_null_data" - es la base de datos modificada al quitarle aquellas filas con datos nulos o ausentes
"new_data" - es la base de datos de sólo aquellas filas con datos ausentes en las columnas "days_employed" y "total_income"

Todo esto con el fin de revisar si existe alteración en la distribución de los datos

In [17]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts(normalize=True))


secondary education    0.707689
bachelor's degree      0.244367
some college           0.034564
primary education      0.013101
graduate degree        0.000279
Name: education, dtype: float64


In [18]:
print(not_null_data['education'].value_counts(normalize=True))

secondary education    0.707612
bachelor's degree      0.243708
some college           0.034882
primary education      0.013488
graduate degree        0.000310
Name: education, dtype: float64


In [19]:
print(new_data['education'].value_counts(normalize=True))

secondary education    0.708372
bachelor's degree      0.250230
some college           0.031739
primary education      0.009660
Name: education, dtype: float64


Se puede observar que la educación tiene una distribución parecida entre las 3 data frames.Con la excepción que no existen "graduate_degrees" en el data frame de sólo valores ausentes.

Ahora compararemos el estado familiar:

In [20]:
print(data['family_status'].value_counts(normalize=True))

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


In [21]:
print(not_null_data['family_status'].value_counts(normalize=True))

married              0.575836
civil partnership    0.193013
unmarried            0.130484
divorced             0.055966
widow / widower      0.044701
Name: family_status, dtype: float64


In [22]:
print(new_data['family_status'].value_counts(normalize=True))

married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64


Como se puede observar la distribución de datos es muy parecida entre los 3 dataframes.

Ahora compararemos el tipo de ingreso:

In [23]:
print(data['income_type'].value_counts(normalize=True))

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
entrepreneur                   0.000093
unemployed                     0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64


In [24]:
print(not_null_data['income_type'].value_counts(normalize=True))

employee                       0.517493
business                       0.236525
retiree                        0.177924
civil servant                  0.067800
unemployed                     0.000103
entrepreneur                   0.000052
student                        0.000052
paternity / maternity leave    0.000052
Name: income_type, dtype: float64


In [25]:
print(new_data['income_type'].value_counts(normalize=True))

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64


Como se puede observar la distribución es muy parecida entre "data" y "not_null_dat". Sin embargo, nos percatamos que aquellas filas con datos ausentes en ingresos y dias de empleo es inexistente para las categorías de "income_type" cuando son "unemployed", "student" y "paternity / maternity leave".

Ahora compararemos el número de hijos:

In [26]:
print(data['children'].value_counts(normalize=True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


In [27]:
print(not_null_data['children'].value_counts(normalize=True))

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64


In [28]:
print(new_data['children'].value_counts(normalize=True))

 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64


Podemos percatarnos que las distribuciones son parecidas entre los 3 dataframes. También detectamos valores incongruentes como un alto número de hijos (20) y un número de hijos negativo. Que puede deberse a un error de tipeo.

Ahora compararemos el género:

In [29]:
print(data['gender'].value_counts(normalize=True))

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64


In [30]:
print(not_null_data['gender'].value_counts(normalize=True))

F      0.658984
M      0.340964
XNA    0.000052
Name: gender, dtype: float64


In [31]:
print(new_data['gender'].value_counts(normalize=True))

F    0.682613
M    0.317387
Name: gender, dtype: float64


Como podemos observar las distribuciones son parecidas entre los data frames. Sin embargo, en aquellas filas con valores ausentes en días empleados y en ingresos totales, no existe el género XNA.

Ahora compararemos las deudas:

In [32]:
print(data['debt'].value_counts(normalize=True))

0    0.919117
1    0.080883
Name: debt, dtype: float64


In [33]:
print(not_null_data['debt'].value_counts(normalize=True))

0    0.918816
1    0.081184
Name: debt, dtype: float64


In [34]:
print(new_data['debt'].value_counts(normalize=True))

0    0.921803
1    0.078197
Name: debt, dtype: float64


Se observa que la mayoría de la muestra paga a tiempo sus deudas. Las distribuciones son parecidas entre los 3 dataframes. 

## Transformación de datos
A continuación se realizará la transformación de ciertos datos que dificultan la tabulación y procesamiento de la información. 


Como se puede observar existen 5 categorías en nivel de educación

A continuación se arreglará los valores de la columna Children debido a ciertos datos imposibles como el -1 niño y un dato excesivo como los 20 niños. 

In [35]:
# Veamos la distribución de los valores en la columna `children`
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [36]:
# [arregla los datos según tu decisión]
#Se asume que existió un typo al ingresar el número de niños. Es decir si dice menos 1 en realidad es un niño, y 20 en realidad son 2
data.loc[data['children']==-1, 'children']= 1
data.loc[data['children']==20, 'children']= 2


In [37]:
data['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5])

In [38]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
data['days_employed'].sort_values().unique()

array([-18388.94990057, -17615.56326563, -16593.47281726, ...,
       401715.81174889, 401755.40047533,             nan])

A continuación se detalla los problemas encontrados en la columna 'days employed':

1. Valores positivos muy altos
2. Valores negativos
3. Valores ausentes

In [39]:

data.loc[data['days_employed']>=0, 'days_employed'] = np.nan

In [40]:
# Aborda los valores problemáticos, si existen.
data['days_employed']= data['days_employed'].abs()


In [41]:
data['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

In [ ]:
Se realizó una limpieza de datos a la columna 

In [42]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [43]:
# Resuelve los problemas en la columna `dob_years`, si existen
data = data[~(data['dob_years'] == 0)]

In [44]:
# Comprueba el resultado - asegúrate de que esté arreglado
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [45]:
# Veamos los valores de la columna


data['family_status'].sort_values().unique()

array(['civil partnership', 'divorced', 'married', 'unmarried',
       'widow / widower'], dtype=object)

In [46]:
# Veamos los valores en la columna
data['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

In [47]:
# Aborda los valores problemáticos, si existen
data = data[~(data['gender'] == 'XNA')]

In [48]:
# Comprueba el resultado - asegúrate de que esté arreglado

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [49]:
# Veamos los valores en la columna
data['income_type'].sort_values().unique()

array(['business', 'civil servant', 'employee', 'entrepreneur',
       'paternity / maternity leave', 'retiree', 'student', 'unemployed'],
      dtype=object)

In [50]:
# Comprobar los duplicados
data.duplicated().sum()


71

In [51]:
# Aborda los duplicados, si existen
data = data.drop_duplicates().reset_index(drop=True)

In [52]:
# Última comprobación para ver si tenemos duplicados
data.duplicated().sum()

0

In [53]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
data.shape

(21352, 12)

# Trabajar con valores ausentes

In [54]:
# Encuentra los diccionarios


In [55]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

In [56]:
data['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [57]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [58]:
data['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

A continuación se crea el diccionario de la educación, donde la clave es la variable "education_id" y el valor es la variable "education"

In [59]:
dictionary_education = {0:["bachelor's degree"],
                        1:['secondary education'],
                        2:['some college'],
                        3:['primary education'],
                        4:['graduate degree']}

In [60]:
dictionary_education

{0: ["bachelor's degree"],
 1: ['secondary education'],
 2: ['some college'],
 3: ['primary education'],
 4: ['graduate degree']}

A continuación se crea el diccionario de la situación familiar, donde la clave es la variable "family_status_id" y el valor es la variable "family_status"

In [61]:
dictionary_family = {0:['married'],
                        1:['civil partnership'],
                        2:['widow / widower'],
                        3:['divorced'],
                        4:['unmarried']}

In [62]:
dictionary_family 

{0: ['married'],
 1: ['civil partnership'],
 2: ['widow / widower'],
 3: ['divorced'],
 4: ['unmarried']}

### Restaurar valores ausentes en `total_income`

In [63]:
# Vamos a escribir una función que calcule la categoría de edad
def age_group(age):
    """
    The function returns the age group according to the age value, using the following rules:
    —'children' for age <= 18
    —'adult' for 19 <= age <= 64
    —'retired' for all other cases
    """
    
    if age <= 10:
        return 'Generation Alpha'
    if age <= 26:
        return 'Generation Z'
    if age <= 42:
        return 'Millennials'
    if age <= 58:
        return 'Generation X'
    if age <= 77:
        return 'Baby Boomers'
    if age <= 98:
        return 'Silent Generation'
    return 'Greatest Generation'
    

In [64]:
# Prueba si la función funciona bien
print(age_group(100))

Greatest Generation


In [65]:
# Crear una nueva columna basada en la función
data['age_group'] = data['dob_years'].apply(age_group)
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Millennials
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Millennials
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Millennials
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Millennials
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Generation X
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,Millennials
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,Generation X
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,Generation X
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,Millennials
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,Millennials


In [66]:
# Comprobar cómo los valores en la nueva columna
print(data['age_group'].value_counts())


Millennials     9010
Generation X    7760
Baby Boomers    2943
Generation Z    1639
Name: age_group, dtype: int64


In [67]:
debtors = data.groupby('age_group')['debt'].sum()
print(debtors.sort_values(ascending = False))

age_group
Millennials     862
Generation X    540
Generation Z    178
Baby Boomers    153
Name: debt, dtype: int64


In [68]:
income = data.groupby('age_group')['total_income'].sum()
print(income.sort_values(ascending = False))

age_group
Millennials     2.293585e+08
Generation X    1.890879e+08
Baby Boomers    6.208103e+07
Generation Z    3.550075e+07
Name: total_income, dtype: float64


La mayor parte de la Generación de Millenials nacidos entre 1981 y 1996 son aquellos que integran la fuerza laboral y por tanto poseen un empleo. 

In [69]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
data.groupby('age_group')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
age_group,,,,,,,,
Baby Boomers,2665.0,23294.947072,14785.961067,3471.216,13905.76800,20077.643,28328.1940,274402.943
Generation X,6979.0,27093.845206,16725.440926,3306.762,16539.37450,23312.944,33050.6900,362496.645
Generation Z,1485.0,23906.231648,11543.441614,5217.034,16034.23800,21863.538,28828.3850,105400.683
Millennials,8130.0,28211.374678,17343.707596,3392.845,17473.51775,24609.170,34270.5255,352136.354


In [70]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
data.groupby('children')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
children,,,,,,,,
0,12648.0,26429.583171,15946.559349,3306.762,16242.51225,23033.3300,32275.53300,362496.645
1,4372.0,27377.506656,17581.246730,3418.824,17137.75225,23655.3185,32853.93700,352136.354
2,1904.0,27469.471068,17062.960565,4494.861,16379.14800,23127.7930,33498.43875,176552.869
3,293.0,29366.910652,18980.660130,4860.001,17820.00500,25191.6190,36014.01700,174660.414
4,34.0,27289.829647,13353.344830,12624.133,19699.82225,24981.6340,29396.31975,79094.031
5,8.0,27268.847250,12897.376823,7803.663,18234.35525,29816.2255,35773.37825,43050.936


In [71]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
data.groupby('education')['total_income'].describe()


,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
bachelor's degree,4684.0,33172.428387,21749.875599,5148.514,20274.59525,28054.5310,40040.08075,362496.645
graduate degree,6.0,27960.024667,12205.330046,15800.399,18005.02925,25161.5835,38593.85350,42945.794
primary education,261.0,21144.882211,10873.977874,4049.374,13117.13300,18741.9760,27119.02400,78410.774
secondary education,13636.0,24600.353617,13702.755630,3306.762,15624.89375,21839.4075,30224.11025,276204.162
some college,672.0,29035.057865,15657.764603,5514.581,18230.03525,25608.7945,36620.90750,153349.533


In [72]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
data.groupby('family_status')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
family_status,,,,,,,,
civil partnership,3716.0,26702.249322,16567.160905,3392.845,16432.19500,23195.6360,32534.67275,276204.162
divorced,1074.0,27202.683563,16967.724091,5402.850,16609.60850,23584.9695,32929.62475,216039.297
married,11098.0,27045.383530,16720.475575,3306.762,16574.30575,23377.7080,32807.90250,362496.645
unmarried,2510.0,26943.601742,16186.890819,3913.227,16949.92300,23139.4040,32503.59975,274402.943
widow / widower,861.0,23006.808776,12652.278709,5443.908,14155.63100,20523.2670,28178.91700,117616.523


In [73]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
data.groupby('gender')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,12688.0,24664.752169,14466.797301,3306.762,15301.7740,21469.0015,30066.8215,274402.943
M,6571.0,30905.772981,19174.523885,3392.845,19580.9215,26819.5670,36876.7530,362496.645


In [74]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
data.groupby('income_type')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
business,4558.0,32397.307219,20906.959696,4592.450,20121.03600,27563.0285,39026.88000,362496.645
civil servant,1306.0,27361.316126,15524.744346,4672.012,16825.62750,24083.5065,33498.93200,145672.235
employee,9964.0,25824.679592,14626.269346,3418.824,16447.15575,22815.1035,31490.45100,276204.162
entrepreneur,1.0,79866.103000,NaN,79866.103,79866.10300,79866.1030,79866.10300,79866.103
paternity / maternity leave,1.0,8612.661000,NaN,8612.661,8612.66100,8612.6610,8612.66100,8612.661
retiree,3426.0,21939.310393,12837.076256,3306.762,13266.76575,18969.1490,27122.84600,117616.523
student,1.0,15712.260000,NaN,15712.260,15712.26000,15712.2600,15712.26000,15712.260
unemployed,2.0,21014.360500,16152.074628,9593.119,15303.73975,21014.3605,26724.98125,32435.602


In [75]:
data_total_income_mean = data.groupby('dob_years')['total_income'].median()


In [76]:
def fix_total_income(row):
    edad = row['dob_years']
    if pd.isna(row['total_income']):
        return data_total_income_mean[edad]
    else:
        return row['total_income']

In [77]:
data['total_income'] = data.apply(fix_total_income, axis = 1)

In [78]:
# Comprueba si tenemos algún error
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     15831 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  float64
 11  purpose           21352 non-null  object 
 12  age_group         21352 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


In [79]:
data_days_employed_mean = data.groupby('dob_years')['days_employed'].median()

In [80]:
def fix_days_employed(row):
    edad = row['dob_years']
    if pd.isna(row['days_employed']):
        return data_days_employed_mean[edad]
    else:
        return row['days_employed']

In [81]:
data['days_employed'] = data.apply(fix_days_employed, axis = 1)

In [82]:
# Comprueba si tenemos algún error
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  float64
 11  purpose           21352 non-null  object 
 12  age_group         21352 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restaurar valores en `days_employed`

In [83]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

data.groupby('income_type')['days_employed'].describe()


,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
business,5057.0,2081.023306,1957.383501,30.195337,751.723546,1607.509390,2707.080682,17615.563266
civil servant,1451.0,3230.222548,2686.945310,39.954170,1302.524255,2411.845400,4409.283018,15193.032201
employee,11029.0,2275.489834,2207.145152,24.141633,812.494753,1631.939403,2917.701025,18388.949901
entrepreneur,2.0,1395.925729,1237.546674,520.848083,958.386906,1395.925729,1833.464552,2271.003374
paternity / maternity leave,1.0,3296.759962,NaN,3296.759962,3296.759962,3296.759962,3296.759962,3296.759962
retiree,3809.0,2555.051943,718.747553,699.061214,2115.452056,2306.399145,2830.361431,8615.516055
student,1.0,578.751554,NaN,578.751554,578.751554,578.751554,578.751554,578.751554
unemployed,2.0,1734.336725,610.675052,1302.524255,1518.430490,1734.336725,1950.242961,2166.149196


## Clasificación de datos


In [84]:
x = data['total_income'].describe()
print(x)


count     21352.000000
mean      26448.755274
std       15711.119166
min        3306.762000
25%       17209.559500
50%       23431.837000
75%       31321.653000
max      362496.645000
Name: total_income, dtype: float64


In [85]:
def income_level(income):
    if income <= x[4]:
        return 'very low wage'
    if income <= x[5]:
        return 'low wage'
    if income <= x[6]:
        return 'acceptable wage'
    else:
        return 'high wage'

In [86]:
print(income_level(1000000000))

high wage


In [87]:
# Crear una nueva columna basada en la función
data['wage_category'] = data['total_income'].apply(income_level)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,wage_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Millennials,high wage
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Millennials,low wage
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Millennials,low wage
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Millennials,high wage
4,0,2252.537414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Generation X,acceptable wage
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,Millennials,high wage
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,Generation X,high wage
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,Generation X,low wage
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,Millennials,very low wage
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,Millennials,low wage


A continuación se reclasifica los propósitos de préstamos:

In [88]:
def new_purpose(purpose):
    if 'wed' in  purpose:
        return 'have a wedding'
    if 'real' in  purpose:
        return 'buy a house'
    if 'hous' in  purpose:
        return 'buy a house'
    if 'edu' in purpose:
        return 'get education'
    else:
        return 'buy a car'
    return ''

In [89]:
data['purpose_category'] = data['purpose'].apply(new_purpose)
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,wage_category,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Millennials,high wage,buy a house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Millennials,low wage,buy a car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Millennials,low wage,buy a house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Millennials,high wage,get education
4,0,2252.537414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Generation X,acceptable wage,have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,Millennials,high wage,buy a house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,Generation X,high wage,buy a house
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,Generation X,low wage,get education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,Millennials,very low wage,have a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,Millennials,low wage,buy a house


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [90]:
# Comprueba los datos sobre los hijos y los pagos puntuales
# Calcular la tasa de incumplimiento en función del número de hijos
data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean']) 




,count,sum,mean
,debt,debt,debt
children,,,
0,14021,1058,0.075458
1,4839,442,0.091341
2,2114,202,0.095553
3,328,27,0.082317
4,41,4,0.097561
5,9,0,0.000000


**Conclusión**
Aquellas personas con 2 o 1 hijo suelen tener una mayor tasa de incumplimiento de pagos puntuales.



**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [91]:
# Comprueba los datos del estado familiar y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el estado familiar

data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean']) 

,count,sum,mean
,debt,debt,debt
family_status,,,
civil partnership,4129,386,0.093485
divorced,1185,85,0.071730
married,12290,927,0.075427
unmarried,2794,273,0.097709
widow / widower,954,62,0.064990


**Conclusión**
En conclusión, aquellas personas sin casarse suelen tener mayor posibilidad de incumplir con el pago a tiempo, debido a la falta de compromiso.


**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [92]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el nivel de ingresos

data.pivot_table(index='wage_category', values='debt', aggfunc=['count', 'sum', 'mean']) 

,count,sum,mean
,debt,debt,debt
wage_category,,,
acceptable wage,5338,475,0.088985
high wage,5338,382,0.071562
low wage,5338,451,0.084489
very low wage,5338,425,0.079618


**Conclusión**
Aquellas personas con un bajo nivel salarial suelen tener mayor incumplimiento de los pagos a tiempo. 


**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [93]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean']) 


,count,sum,mean
,debt,debt,debt
purpose_category,,,
buy a car,7307,632,0.086492
buy a house,8233,590,0.071663
get education,3502,327,0.093375
have a wedding,2310,184,0.079654


Como se observa, existe un mayor incumplimiento del pago de deuda por parte de aquellos que prestaron dinero para educación y comprar un automóvil

In [94]:
data.pivot_table(index='age_group', values='debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
age_group,,,
Baby Boomers,2943,153,0.051988
Generation X,7760,540,0.069588
Generation Z,1639,178,0.108603
Millennials,9010,862,0.095671


Como se observa, existe un mayor incumplimiento del pago de deuda por parte de la Generación Z y la generación de los Millenials. 

# Conclusión general 
En conclusión, existe mayor potencial de pago de deuda a tiempo por parte de la generación de Baby Boomers, viudos y sin hijos. 
Mientras es menos tentador prestar a aquellos que mayormente incumplen como la generación Z y Millenials, aquellos que prestan por educación, aquellos no ca


A partir del procesamiento y análisis de datos se encontro los siguientes hallazgos:
1. Se realizó la limpieza de datos en las columnas "days_employed" mediante la eliminación de aquellos datos donde existía una cantidad exagerada de días trabajados como máximo que puede laborar una persona. 
2. Se corrigió las inconsistencia de datos en la columna "children" dado que existía valores negativos que pudieron ocurrir debido a un error de tipeo y cantidad exagerada de hijos (20).
3. Se recategorizó el propósito por el cual las personas se endeudan o piden un préstamo, mediante la identificación de patrones. 
4. Se crearon dos diccionarios donde se aborda la educación y la situación familiar. 
5. Se realizó categorización de la población por grupo generacional, en base a la edad. 
6. Se comparó la distribución de los datos, de forma que se constató que las distribuciones no sean afectadas por aquellos datos ausentes.
7. Se completó los datos ausentes en "total_income" mediante el uso de la función "fix_total_income" la cual considera la agrupación de la mediana de los ingresos por edad del cliente. 
8. Se completó los datos ausentes en las columna "days_employed" mediante la eliminación de filas con datos de días de trabajo que superen los días máximos de trabajo posible. 


En conclusión se logró el objetivo de preparar un informe para la división de préstamos de un banco. 
Basado en el análisis se determinó que aquellas personas "unmarried" suelen tener mayor posibilidad de incumplir con el pago a tiempo, debido a la falta de compromiso, y que aquellas personas con 2 o 1 hijo suelen tener una mayor tasa de incumplimiento de pagos puntuales. Estos hallazgos permitirán crear una puntuación de crédito para un cliente potencial y evaluar la capacidad de un prestatario potencial para pagar su préstamo.

Lo más conveniente es realizar préstamo bancario a perfiles como: 
1. Personas sin hijos, viudos o divorciados, con un alto salario, con el propósito de adquirir una casa y que sea de la generación X o Baby Boomer. 